# STL-Embedding ROPC & AOV

In [ ]:
import libs.config as config
import libs.hyperparam as hyperparam
import libs.util as util 
import libs.customized_dataset as customized_dataset
import libs.models as models 
import libs.forecasting as forecasting 
import libs.plots as plots 

import os  # for interacting with the operating system
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # for debugging cuda errors
import glob  # for finding files in directories
import warnings 
warnings.filterwarnings('ignore') # for ignoring all warnings
# import argparse  # for parsing command line arguments (for running from the terminal)

import random
import math # for math operations
import time  # for time-related functionalities
import holidays # for checking if a date is a holiday
import matplotlib.pyplot as plt  # for plotting
plt.set_cmap('cividis') # color map for the plots to 'cividis'
## Enable the display of matplotlib plots inline in a Jupyter notebook
%matplotlib inline 
import matplotlib.ticker as ticker # for customizing the plots' tick locations and labels
import numpy as np  # for numerical computations
import pandas as pd  # for data manipulation and analysis
pd.set_option('display.max_columns', None) # to display all columns
from datetime import date, datetime, timedelta  # for working with dates and times

from sklearn.preprocessing import MinMaxScaler, RobustScaler, Normalizer  # for scaling data
from tqdm import tqdm  # for creating progress bars

import torch  # for building and training neural networks
from torch.utils.data import Dataset, DataLoader # for loading and managing datasets
import torch.nn as nn  # for building neural networks
import torch.nn.functional as F  # for implementing various activation functions
import torch.optim as optim  # for defining optimizers

# set the device as GPU with index 0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
config.get_environ_info(device)

### Check NVIDIA GPU

In [ ]:
# !nvidia-smi

### Set data path

In [ ]:
## Google cluster
# READ_DIR = r"/home/kkim476/dl-cbcv/data/weekly_cohort_data_1000_missing_filled_final"
READ_DIR = r"/home/kkim476/dl-cbcv/data/selected_ten"
WORK_DIR = r"/home/kkim476/dl-cbcv"


### Set prediction target, covariates, and save mode

In [ ]:
# set prediction target
PREDICTION_GOAL = 'repeat_order_per_customer' 
TARGET_TASK = config.get_target_variable_name(PREDICTION_GOAL)

### set covariate features
USE_EMBEDDING = True
COHORT_EMBEDDING = True
DUMMY_VAR = False # entity embedding, rather than one hot encoding

### folder path for saving results
SAVE_MODE = False
SAVE_DIR = f'{WORK_DIR}/results/stl_embedding_{PREDICTION_GOAL}_{datetime.today().strftime("%Y-%m-%d")}'
SAVE_MODEL, SAVE_EPOCH, SAVE_PLOT, SAVE_PREDICT, SAVE_ACTUAL = config.create_save_folders(SAVE_MODE, SAVE_DIR)

## Data Preparation

### Load data

In [ ]:
## this file is cohort-week level aggregated panel data after ETL of raw earnest transaction DB
raw_df, MERCHANT_NAMES_EMB_INT = util.read_files_generate_behaviorfeatures_get_embed_dict(
    READ_DIR, hyperparam.TRAIN_START, hyperparam.TEST_START, hyperparam.TEST_END,
    group_identifier='acq_week', time_identifier='week',
    acquisition_identifier='N_week_cohort',
    order_identifier = 'orders',
    spend_identifier = 'spend')

FREQ, week_start = util.get_week_start(raw_df, hyperparam.TRAIN_START, hyperparam.TEST_START, hyperparam.TEST_END) 

### Zero padding

This is technical but important piece regarding how to handle 'cohort' triangle data.

In [ ]:
# columns that I want to keep
company_static = ['merchant_index', 'merchant', 'parent_merchant', 'category', 'subcategory']
if USE_EMBEDDING:
    company_static = company_static + ['merchant_emb_int', 'merchant_name']
# company_dynamic = ['cohort_size', 'active_users', 
#                    'orders','rpt_orders','initial_order',
#                    'spend','rpt_spend','initial_spend',
#                    'initial_aov','rpt_aov',
#                    ]
company_dynamic = []

assert all(column in list(raw_df.columns) for column in company_static + company_dynamic), \
    'some columns are missing in the raw_df'

df_padded = util.zero_padding(raw_df, [TARGET_TASK], company_static, company_dynamic, hyperparam.INPUT_CHUNK_LENGTH, FREQ,
                               use_merchant_embedding=True, merchant_name='merchant_name')


### Generate calendar time covariates

Our covariates include:
- week of year_t+1 (1 week ahead) : (categorical) one-hot encoding. 53 variables.
- holidays_t+1 (1 week ahead) : (binary) dummy variable. 1 variable.
- global trend_t : (continuous) quadratic. 2 variables.
- cohort numbering_i : (continuous) quadratic. 2 variables.
- cohort tenure_it : (continuous) quadratic. 2 variables.

In [ ]:
df_padded_w_cov, country_holidays = util.generate_calendartime_features(df_padded, FREQ, week_start, DUMMY_VAR)
# util.check_holidays(df_padded_w_cov, country_holidays)
df_padded_w_cov = util.generate_cohort_features(df_padded_w_cov, [TARGET_TASK], COHORT_EMBEDDING, DUMMY_VAR, hyperparam.COHORT_EMB_NUM)

In [ ]:
## limit data range
df = df_padded_w_cov[df_padded_w_cov['group']<=hyperparam.TEST_END] # limit cohort
df = df[df['time']>=hyperparam.TRAIN_START_with_offset][(df['time']<=hyperparam.TEST_END)] # limit time window

## get covariate feature names
COVARIATE_FEATURE_NAMES, covariate_name_to_index = config.get_covariate_variable_name(df, USE_EMBEDDING, COHORT_EMBEDDING, DUMMY_VAR)
print(COVARIATE_FEATURE_NAMES)
df

In [ ]:
del df_padded
del df_padded_w_cov
del country_holidays

### Split into train:val:test (time-wise) X censored:uncensored (group-wise)


In [ ]:
(main_df, main_df_train, main_df_valid, main_df_test, 
 censored_df, censored_df_train, censored_df_valid, censored_df_test) = util.split_dataframe(
    df, hyperparam.TRAIN_START, hyperparam.VAL_START, hyperparam.TEST_START, 
    hyperparam.VAL_START_with_offset, hyperparam.TEST_START_with_offset, hyperparam.VAL_LOSS)

### Transform data frame to scaled numpy array

1. As numpy array is more efficient to handle, we transform pandas data frame into numpy array.

2. For each task, we scale the data with its own scaler. This is important for multi-task learning as each task has different scale.

In [ ]:
## entire sequences (unscaled)
whole_dict = util.df_to_numpy(df, TASKS=[TARGET_TASK], COVARIATES=COVARIATE_FEATURE_NAMES, 
                              use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')

## main cohorts' sequences
main_train_dict = util.df_to_scaled_numpy(main_df_train, [TARGET_TASK], COVARIATE_FEATURE_NAMES,
                                          use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')
main_test_dict = util.df_to_scaled_numpy(main_df_test, [TARGET_TASK], COVARIATE_FEATURE_NAMES,  main_train_dict['scaler'],
                                         use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')
if hyperparam.VAL_LOSS:
    main_val_dict = util.df_to_scaled_numpy(main_df_valid, [TARGET_TASK], COVARIATE_FEATURE_NAMES, main_train_dict['scaler'],
                                            use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')

## censored cohorts' sequences
censored_train_dict = util.df_to_scaled_numpy(censored_df_train, [TARGET_TASK], COVARIATE_FEATURE_NAMES, main_train_dict['scaler'],
                                              use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')
censored_test_dict = util.df_to_scaled_numpy(censored_df_test, [TARGET_TASK], COVARIATE_FEATURE_NAMES, main_train_dict['scaler'],
                                             use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')
if hyperparam.VAL_LOSS:
    censored_val_dict = util.df_to_scaled_numpy(censored_df_valid, [TARGET_TASK], COVARIATE_FEATURE_NAMES, main_train_dict['scaler'],
                                                use_merchant_embedding=USE_EMBEDDING, merchant_name='merchant_name')


## Dataset and DataLoader
the creation of custom datasets, and the initialization of the DataLoader.
- custom dataset class will group the data by the group column
- collate function will handle the padding and attention masking

<img src="../img/data%20period%20and%20samples.png" alt="data_period" width="600" height="400">

<img src="../img/input%20output%20format.png" alt="input_output" width="600" height="400">



### Generate train loader and validation loader

In [ ]:
from libs.customized_dataset import CrossSectionalTimeSeriesDataset, collate_fn, value_dict_to_np
import multiprocessing

## Create a TimeSeriesDataset instance and initialize DataLoader for each data
value_train = {key: main_train_dict['scaled_value_seq_dict'].get(key, []) +\
  censored_train_dict['scaled_value_seq_dict'].get(key, []) \
    for key in set(main_train_dict['scaled_value_seq_dict']) | set(censored_train_dict['scaled_value_seq_dict'])}
cov_train = main_train_dict['cov_seq'] + censored_train_dict['cov_seq']
value_train_np = value_dict_to_np(value_train, [TARGET_TASK]) # shape: (num_groups, seq_len, num_tasks)
train_dataset = CrossSectionalTimeSeriesDataset(value_train_np, cov_train, hyperparam.INPUT_CHUNK_LENGTH)

## To be iterated over batches of data during training
train_loader = DataLoader(train_dataset,
                          batch_size=hyperparam.BATCH_SIZE, # how many samples per batch to load
                          shuffle=True, # have the data reshuffled at every epoch to reduce model overfitting
                          drop_last=False, # If False and the size of dataset is not divisible by the batch size, then the last batch will be smaller
                          collate_fn=collate_fn,
                          pin_memory=True, # True for faster data transfer to GPUs, False if out of memory
                          num_workers=multiprocessing.cpu_count()//2,
                        #   num_workers=hyperparam.NUM_WORKERS, # how many subprocesses to use for data loading. (0: loaded in the main process)
                          )

if hyperparam.VAL_LOSS:
    value_valid = {key: main_val_dict['scaled_value_seq_dict'].get(key, []) +\
        censored_val_dict['scaled_value_seq_dict'].get(key, []) \
            for key in set(main_val_dict['scaled_value_seq_dict']) | set(censored_val_dict['scaled_value_seq_dict'])}
    cov_valid = main_val_dict['cov_seq'] + censored_val_dict['cov_seq']
    value_valid_np = value_dict_to_np(value_valid, [TARGET_TASK]) # shape: (num_groups, seq_len, num_tasks)
    val_dataset = CrossSectionalTimeSeriesDataset(value_valid_np, cov_valid, hyperparam.INPUT_CHUNK_LENGTH)
    val_loader = DataLoader(val_dataset, batch_size=hyperparam.BATCH_SIZE, shuffle=False, drop_last=False, # no shuffle for validation
                              collate_fn=collate_fn,
                              )

## get the dimension of the target and covariate data
first_sample = next(iter(train_dataset)) # or train_dataset[0]
first_batch = next(iter(train_loader))

## number of targets, number of covariate features
tgt_dim, cov_dim = first_sample["target"].shape[1], first_sample["covariate"].shape[1] # 3 , 60


## Model

If considering attention mask later, modify with this:
- `def forward(self, src, attention_mask):`
- `x = self.transformer(src=src, tgt=tgt, src_key_padding_mask=attention_mask)`

## Training (Estimation)

In [ ]:
## For memory monitoring ==
# !pip install memory_profiler
# %load_ext memory_profiler
# %memit my_function()

In [ ]:
from libs.models import STL_Transformer

## initialize model
model = STL_Transformer(
    input_dim=tgt_dim + cov_dim,
    feature_dict=covariate_name_to_index, 
    num_merchant=len(MERCHANT_NAMES_EMB_INT),
).to(device)

## define optimizer and loss criterion
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparam.LEARNING_RATE)
loss_criterion = torch.nn.MSELoss()

## initialize empty list for losses and early stop
train_losses, valid_losses = [], []
pre_valid_loss, cnt_no_improve = np.inf, 0

In [ ]:
for epoch in tqdm(range(hyperparam.N_EPOCHS), desc="training", unit="epoch"):
    train_loss = 0.0 # within each epoch, initialize train loss to 0

    for target_input, cov_input, gt in train_loader:
        optimizer.zero_grad() # reset optimizer gradients to zero
        train_input = torch.cat((target_input,cov_input), dim=-1).to(device)   
        next_value = model(train_input) # forward pass data through the model
        loss = loss_criterion(next_value, gt.to(device)) # calculate and update loss 
        train_loss += loss.item() # accumulate batch loss within each epoch
        loss.backward() # backpropagation
        _ = nn.utils.clip_grad_norm_(model.parameters(), hyperparam.GRADCLIP) # clip gradients to prevent exploding gradients
        optimizer.step() # update parameters based on gradients

    train_losses.append(train_loss/len(train_loader)) # append total train loss for each epoch

    if hyperparam.VAL_LOSS:
        valid_loss = 0.0
        for target_input, cov_input, gt in val_loader:
            valid_input = torch.cat((target_input,cov_input), dim=-1).to(device)
            next_value = model(valid_input)
            loss = loss_criterion(next_value, gt.to(device))
            valid_loss += loss.item()

        valid_losses.append(valid_loss/len(val_loader))
        
        # Early stop evaluate
        if pre_valid_loss - valid_loss  < hyperparam.MINDELTA:
            cnt_no_improve += 1
            if cnt_no_improve > hyperparam.PATIENCE:
                break
        else:
            cnt_no_improve = 0
            
        pre_valid_loss = valid_loss
    
    if epoch % 5 == 0:
        print("train_loss:{:.4f}".format(train_loss))
        print("val loss: {:.4f}".format(valid_loss))

In [ ]:
def plot_losses():
    plt.figure(figsize=(10, 6))
    plt.xlabel("# of epoch")
    plt.plot(train_losses, label="train loss")
    plt.plot(valid_losses, label="valid loss")
    plt.title(f"{PREDICTION_GOAL} Embedding {len(MERCHANT_NAMES_EMB_INT)} model Losses")
    plt.legend()
        
plot_losses()

In [ ]:
import pickle

if SAVE_MODE:
    # To save the model
    with open(f'{SAVE_MODEL}/{PREDICTION_GOAL}_embedding_{len(MERCHANT_NAMES_EMB_INT)}.pkl', 'wb') as file:
        pickle.dump(model, file)

    # # To load the model
    # with open(f'{SAVE_MODEL}/{PREDICTION_GOAL}_embedding_{len(MERCHANT_NAMES_EMB_INT)}.pkl', 'rb') as file:
    #     model = pickle.load(file)


## Prediction (Inference)

- Rolling forecast origin or walk-forward validation (which means generating predictions one step at a time and conditioning upon the predicted values)

For each rolling window:
- Use the last `INPUT_CHUNK_LENGTH` weeks of data as input to forecast the next week.
- Append the forecasted value to the actual data.
- Move the window one week forward and repeat.

*NOTE:* Error can be accumulated in **triple** way as we now take acq_hat, repeat order per customer_hat, aov_hat all together for next acq prediction,for example.

### Cohort 1 (acquired after the beginning of train period)

In [ ]:
from libs.forecasting import prepare_TimeSeriesDataset, rolling_forecast_stl

# Sets the module in evaluation mode
model.eval()

# rolling forecast (scaled)
main_value_test_np = value_dict_to_np(main_test_dict['scaled_value_seq_dict'], [TARGET_TASK])
test_datasets = prepare_TimeSeriesDataset(main_value_test_np, main_test_dict['cov_seq'], 
                                          CrossSectionalTimeSeriesDataset, hyperparam.INPUT_CHUNK_LENGTH)
main_value_test_pred = rolling_forecast_stl(test_datasets, model, device, TARGET_TASK)

# get ground truth in the test period
main_df_test_net = main_df_test[main_df_test['time']>=hyperparam.TEST_START][main_df_test['tenure']>=0]
actual_main = main_df_test_net[['group', 'time'] + [TARGET_TASK]]

# scale back the forecast
predicted_main = util.inverse_scale_np_to_dataframe_embedding(main_test_dict['scaler'], main_test_dict['group_seq'], main_test_dict['time_seq'], 
                                                                  main_value_test_pred, hyperparam.INPUT_CHUNK_LENGTH)
predicted_main.sort_values(['group', 'time'], inplace=True)

In [ ]:
from libs.plots import plot_time_series_multiple

def plot_ropc_cohort1(merchant_name):
    selected_tuples = [t for t in main_test_dict['group_seq'] if t[0] == merchant_name]
    fig, axs = plt.subplots(3, 1, figsize=(20, 20))  # 3 rows, 1 column
    selected_group_indices = random.sample(range(len(selected_tuples)), 3)
    for i, group_index in enumerate(selected_group_indices):
        group = selected_tuples[group_index][1]
        plot_time_series_multiple(
            main_df[(main_df['group'] == group) & (main_df['merchant_name'] == merchant_name)], 
            predicted_main[(predicted_main['group'] == group) & (predicted_main['merchant_name'] == merchant_name)],
            'time', TARGET_TASK, 
            title=f'{merchant_name} [Group {group}\'s Repeat Order per Customer]',
            ax=axs[i]
        )
    plt.tight_layout()

selected_merchant_name = list(MERCHANT_NAMES_EMB_INT.keys())[0]  # assuming MERCHANT_NAMES_EMB_IDX is a dictionary
plot_ropc_cohort1(selected_merchant_name)

### Cohort 0 (acquired before the beginning of train period)

In [ ]:
# Sets the module in evaluation mode
model.eval()

# rolling forecast (scaled)
censored_value_test_np = value_dict_to_np(censored_test_dict['scaled_value_seq_dict'], [TARGET_TASK])
censored_test_datasets = prepare_TimeSeriesDataset(censored_value_test_np, censored_test_dict['cov_seq'], 
                                                   CrossSectionalTimeSeriesDataset, hyperparam.INPUT_CHUNK_LENGTH)
censored_value_test_pred = rolling_forecast_stl(censored_test_datasets, model, device, TARGET_TASK)

# get ground truth in the test period
censored_df_test_net = censored_df_test[censored_df_test['time']>=hyperparam.TEST_START][censored_df_test['tenure']>=0]
actual_censored = censored_df_test_net[['merchant_name', 'group', 'time'] + [TARGET_TASK]]

# scale back the forecast
predicted_censored = util.inverse_scale_np_to_dataframe_embedding(censored_test_dict['scaler'], censored_test_dict['group_seq'], censored_test_dict['time_seq'], 
                                                                  censored_value_test_pred, hyperparam.INPUT_CHUNK_LENGTH)
predicted_censored.sort_values(['group', 'time'], inplace=True)

#### Repeat order per customer plot

In [ ]:
def plot_ropc_cohort0(merchant_name):
    selected_tuples = [t for t in censored_test_dict['group_seq'] if t[0] == merchant_name]
    fig, axs = plt.subplots(3, 1, figsize=(20, 20))  # 3 rows, 1 column
    selected_group_indices = random.sample(range(len(selected_tuples)), 3)
    for i, group_index in enumerate(selected_group_indices):
        group = selected_tuples[group_index][1]
        plot_time_series_multiple(
            actual_censored[(actual_censored['group'] == group) & (actual_censored['merchant_name'] == merchant_name)], 
            predicted_censored[(predicted_censored['group'] == group) & (predicted_censored['merchant_name'] == merchant_name)],
            'time', TARGET_TASK, 
            title=f'{merchant_name} [Group {group}\'s Repeat Order per Customer]',
            ax=axs[i]
        )
    plt.tight_layout()

selected_merchant_name = list(MERCHANT_NAMES_EMB_INT.keys())[0]  # assuming MERCHANT_NAMES_EMB_IDX is a dictionary
plot_ropc_cohort0(selected_merchant_name)

In [ ]:
from libs.plots import save_plots_to_pdf

if SAVE_MODE:
    # Saving traininig and validation losses
    save_plots_to_pdf([
        (plot_losses, {}),
        ], f'{SAVE_EPOCH}/{PREDICTION_GOAL}_embedding_{len(MERCHANT_NAMES_EMB_INT)}_loss.pdf')
    
    predicted_combined = pd.concat([predicted_censored, predicted_main])
    actual_combined = pd.concat([actual_censored, actual_main])

    for key in MERCHANT_NAMES_EMB_INT.keys():
        MERCHANT_NAME = key    
        predicted_main_sub = predicted_combined[predicted_combined.merchant_name == MERCHANT_NAME]
        predicted_main_sub.to_csv(f'{SAVE_PREDICT}/{MERCHANT_NAME}_pred.csv', index=False)
        actual_combined_sub = actual_combined[actual_combined.merchant_name == MERCHANT_NAME]        
        actual_combined_sub.to_csv(f'{SAVE_PREDICT}/{MERCHANT_NAME}_pred.csv', index=False)
        
        # Saving plots to PDF
        save_plots_to_pdf([
            (plot_ropc_cohort1, {'merchant_name': f'{MERCHANT_NAME}'}),
            (plot_ropc_cohort0, {'merchant_name': f'{MERCHANT_NAME}'}),
            ], f'{SAVE_PLOT}/{MERCHANT_NAME}.pdf')
